<a href="https://colab.research.google.com/github/omkar806/customer_churn_prediction/blob/main/Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip archive.zip

Archive:  archive.zip
  inflating: WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [5]:
df = pd.read_csv('data.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [8]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [10]:
def detect_binary_columns():
  binary_columns = []
  for column in df.columns:
    if df[column].nunique() == 2:
      binary_columns.append(column)
  return binary_columns

binary_columns = detect_binary_columns()
binary_columns

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'PaperlessBilling',
 'Churn']

In [18]:
categorical_columns = [column for column in df.columns if column not in binary_columns]
categorical_columns.remove('customerID')
categorical_columns

['tenure',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges']

In [23]:
label_encoders = {}

processed_df = df.copy().drop('customerID',axis=1)

processed_df['TotalCharges'] = pd.to_numeric(processed_df['TotalCharges'], errors='coerce')
processed_df['TotalCharges'].fillna(processed_df['MonthlyCharges'], inplace=True)

for column in categorical_columns:
  processed_df[column] = LabelEncoder().fit_transform(processed_df[column])

for cols in binary_columns:
  processed_df[cols] = processed_df[cols].map({'Yes':1 , 'No':0})

processed_df

<ipython-input-23-31828a29db58>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df['TotalCharges'].fillna(processed_df['MonthlyCharges'], inplace=True)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,NaN,NaN,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,142,74,0
1,NaN,NaN,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,498,3628,0
2,NaN,NaN,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,436,540,1
3,NaN,NaN,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,266,3574,0
4,NaN,NaN,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,729,678,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,NaN,NaN,1,1,24,1,2,0,2,0,2,2,2,2,1,1,3,991,3704,0
7039,NaN,NaN,1,1,72,1,2,1,0,2,2,0,2,2,1,1,1,1340,6308,0
7040,NaN,NaN,1,1,11,0,1,0,2,0,0,0,0,0,0,1,2,137,1269,0
7041,NaN,NaN,1,0,4,1,2,1,0,0,0,0,0,0,0,1,3,795,1161,1
